<a href="https://colab.research.google.com/github/strctlybusines1/lineup/blob/main/Copy_of_SimsVersion2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import random
import numpy as np
import pandas as pd
import urllib, json
import re
from itertools import permutations

from pulp import *

In [3]:
PATH = '/content/NFLW4.csv'

In [4]:
df = pd.read_csv('NFLW4.csv').set_index("Player")

In [5]:
dfPlayerCol = df.reset_index()
df.head()

,Pos,Salary,Team,Opp,STD,Ceiling,Own,Mean
Player,,,,,,,,
Cardinals,DST,2800,ARI,CAR,5.507571,7.0,0.065,6.8
Falcons,DST,2800,ATL,CLE,6.429101,12.0,0.006,4.6
Ravens,DST,2500,BAL,BUF,2.309401,11.0,0.021,4.4
Bills,DST,3200,BUF,BAL,8.504901,18.8,0.042,6.2
Panthers,DST,2900,CAR,ARI,7.937254,11.2,0.039,6.4


In [6]:
df = df[['Pos', 'Salary', 'Team', 'Opp', 'STD', 'Ceiling', 'Own', 'Mean']]

In [7]:
df.head()

,Pos,Salary,Team,Opp,STD,Ceiling,Own,Mean
Player,,,,,,,,
Cardinals,DST,2800,ARI,CAR,5.507571,7.0,0.065,6.8
Falcons,DST,2800,ATL,CLE,6.429101,12.0,0.006,4.6
Ravens,DST,2500,BAL,BUF,2.309401,11.0,0.021,4.4
Bills,DST,3200,BUF,BAL,8.504901,18.8,0.042,6.2
Panthers,DST,2900,CAR,ARI,7.937254,11.2,0.039,6.4


In [8]:
mu = df.iloc[:,7]
sigma = df.iloc[:,4]

In [9]:
r = []
for x in df:
  r = np.random.normal(df['Mean'], df['STD'], (10000, len(df))).T

pd.DataFrame(r)

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,5.754318,6.546583,-3.812016,13.833648,12.414907,7.478537,12.933097,4.333358,12.431752,14.831217,...,13.470149,18.366642,-2.016053,10.350591,6.338235,12.416071,2.888989,-1.518905,2.121883,5.535822
1,-5.569473,13.662614,-1.675802,7.661753,4.317718,15.271682,-0.638366,-0.648001,1.615267,1.827781,...,5.511962,14.715098,1.838490,1.542703,6.270412,1.693443,15.509616,9.255963,2.722539,-5.449078
2,4.721498,7.325271,7.338457,-1.014368,3.852215,7.058582,5.427773,3.744456,9.988054,0.223413,...,-0.593429,6.741949,5.543808,0.114449,5.923907,3.000805,4.783917,0.426507,6.869304,0.480278
3,16.036468,10.029859,2.920440,19.469312,5.827974,-5.105536,-7.094155,11.274256,-9.135465,-1.615394,...,8.016856,-9.999436,1.706937,4.002894,4.022807,4.918579,7.363009,8.323394,7.594610,0.720945
4,8.214848,10.124089,12.106704,1.800285,-2.463006,10.761349,9.644625,15.252706,7.254446,2.260254,...,10.304464,6.666294,0.502367,2.900901,14.654835,0.428588,7.074616,2.661909,6.747455,9.164702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
591,5.038090,1.658014,-7.157385,-8.200716,6.007094,-3.460336,0.669271,3.085487,-5.500536,5.211928,...,-0.914817,4.545891,0.398308,-11.437429,-1.423160,-0.924833,-0.551663,9.260672,1.350095,2.642021
592,-0.915393,1.317144,-4.087119,-1.939990,-8.782317,-2.376871,5.092606,-2.376392,-2.407589,-0.881918,...,-1.530737,-0.657565,2.969672,3.164366,-1.652178,-1.020106,-0.828901,-2.212326,-0.468369,-3.028435
593,1.589761,0.283816,-0.543085,0.695752,-1.370301,0.117414,1.607592,2.337714,0.197031,0.982116,...,-0.312331,1.349920,-1.050879,0.030721,1.804825,0.477666,-0.038112,0.608201,1.047672,1.003481


In [10]:
r_df = pd.DataFrame(r)

In [11]:
df1 = pd.concat([r_df, dfPlayerCol], ignore_index=False)

In [12]:
df1

,0,1,2,3,4,5,6,7,8,9,...,9999,Ceiling,Mean,Opp,Own,Player,Pos,STD,Salary,Team
0,5.754318,6.546583,-3.812016,13.833648,12.414907,7.478537,12.933097,4.333358,12.431752,14.831217,...,5.535822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-5.569473,13.662614,-1.675802,7.661753,4.317718,15.271682,-0.638366,-0.648001,1.615267,1.827781,...,-5.449078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.721498,7.325271,7.338457,-1.014368,3.852215,7.058582,5.427773,3.744456,9.988054,0.223413,...,0.480278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16.036468,10.029859,2.920440,19.469312,5.827974,-5.105536,-7.094155,11.274256,-9.135465,-1.615394,...,0.720945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8.214848,10.124089,12.106704,1.800285,-2.463006,10.761349,9.644625,15.252706,7.254446,2.260254,...,9.164702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,DAL,0.0,Marken Michel,"WR, FLEX",NaN,3000.0,WAS
591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.456,0.0,DAL,0.0,Alex Erickson,"WR, FLEX",5.972359,3000.0,WAS
592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.500,0.0,DAL,0.0,Dyami Brown,"WR, FLEX",2.770078,3000.0,WAS
593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.200,0.5,DAL,0.0,Dax Milne,"WR, FLEX",1.118332,3000.0,WAS


In [13]:
pd.DataFrame.to_csv(df1, "SimsW4EXP.csv")

In [32]:
PATH = '/content/SimResults3.csv'

In [33]:
df2 = pd.read_csv('SimResults3.csv')

In [34]:
df2.head()

,Player,Pos,Salary,Team,Opp,Own,Ceiling,Mean,STD,Pro1,...,Pro9991,Pro9992,Pro9993,Pro9994,Pro9995,Pro9996,Pro9997,Pro9998,Pro9999,Pro10000
0,Colts,DST,3000,IND,TEN,0.069,6.0,6.8,0.000000,6.800000,...,6.800000,6.800000,6.800000,6.800000,6.800000,6.800000,6.800000,6.800000,6.800000,6.800000
1,Raiders,DST,2900,LV,DEN,0.029,3.0,6.1,0.000000,6.100000,...,6.100000,6.100000,6.100000,6.100000,6.100000,6.100000,6.100000,6.100000,6.100000,6.100000
2,Jets,DST,2600,NYJ,PIT,0.037,4.0,6.2,1.154701,4.734930,...,7.586273,5.593391,5.389510,5.247420,6.569658,7.609694,7.802346,7.913406,5.771833,5.500033
3,Seahawks,DST,2400,SEA,DET,0.012,8.2,4.5,1.154701,6.005637,...,5.146496,4.869117,3.849923,4.649844,5.701750,5.661077,4.163328,6.029001,2.867487,3.419947
4,Texans,DST,2400,HOU,LAC,0.025,8.2,4.9,1.527525,4.492400,...,8.314133,5.203631,7.647210,5.406506,3.057599,4.714926,5.450922,5.738930,5.283617,6.891142


In [35]:
pro = df2.iloc[:,9:9999]

In [36]:
flex = df2[df2.Pos.isin(["RB", "WR", "TE"])].copy()
flex.position = "FLEX"
current = pd.concat([df2, flex])

In [37]:
import openpyxl

In [38]:
wb = openpyxl.Workbook()
ws = wb.active

In [39]:
availables = df2.groupby(["Player", "Pos", "Salary", "Team", "Opp", "Own", "Pro1"]).agg('count')
availables = df2.reset_index()

In [48]:
salaries = {}
pro = {}
teams = {}
opp = {}
own = {}

for pos in df2.Pos.unique():
  available_pos = df2[df2.Pos == pos]
  salary = list(available_pos[["Player", "Salary"]].set_index("Player").to_dict().values())[0]
  proj = list(available_pos[["Player", "Pro1"]].set_index("Player").to_dict().values())[0]
  team = list(available_pos[["Player", "Team"]].set_index("Player").to_dict().values())[0]
  opps = list(available_pos[["Player", "Opp"]].set_index("Player").to_dict().values())[0]
  owner = list(available_pos[["Player", "Own"]].set_index("Player").to_dict().values())[0]
  salaries[pos] = salary
  pro[pos] = proj
  team[pos] = teams
  opps[pos] = opp
  owner[pos] = own

pos_num_available = {
    "QB": 1,
    "RB": 2,
    "WR": 3,
    "TE": 2,
    "FLEX": 1,
    "DST": 1
} 

salary_cap = 50000

In [49]:
for lineup in range(1,51):
  _vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in pro.items()}

  prob = LpProblem("Fantasy", LpMaximize)
  rewards = []
  costs = []
  position_constraints = []

  for k, v in _vars.items():
    costs += lpSum([salaries[k][i] * _vars[k][i] for i in v])
    rewards += lpSum([pro[k][i] * _vars[k][i] for i in v])
    prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available[k]

  prob += lpSum(rewards)
  prob += lpSum(costs) <= salary_cap
  if not lineup == 1:
    prob += (lpSum(rewards) <= total_score-0.001)
  prob.solve()

  score= str(prob.objective)
  constraints = [str(const) for const in prob.constraints.values()]
  colnum = 1

  for v in prob.variables():
    score = score.replace(v.name, str(v.varValue))
    if v.varValue !=0:
      ws.cell(row=lineup, column=colnum).value = v.name
      colnum +=1
  total_score = eval(score)
  ws.cell(row=lineup, column=colnum).value = total_score
  print(lineup, total_score)

1 243.44224650000004
2 243.22282637000004
3 242.72271432
4 242.57001699000003
5 242.43352375000003
6 242.28497786000003
7 241.78486580999999
8 241.73162444
9 241.63216848000002
10 241.50580927000001
11 241.49567524000003
12 241.47855426000004
13 241.45287194
14 241.23345181
15 241.00569721999997
16 240.79377592999998
17 240.73333976
18 240.58064242999998
19 240.54070575000003
20 240.49816466000001
21 240.44414919
22 240.29632344
23 240.27874453
24 240.15031555
25 240.13796512000002
26 239.91854499000002
27 239.88103722000002
28 239.84417381
29 239.77863248
30 239.6502035
31 239.62593515
32 239.62475368
33 239.54283488000002
34 239.49750617000004
35 239.48944191
36 239.41843293999997
37 239.38202074
38 239.36101293
39 239.30435728
40 239.26573561
41 239.23413763
42 239.20011661
43 239.12924237000001
44 239.12464163
45 239.08493715
46 239.06500522
47 239.02005224
48 239.0147175
49 238.9990314
50 238.98069648


In [50]:
wb.save("W4lineupsv2.xlsx")

In [28]:
prob = LpProblem("Fantasy", LpMaximize)
rewards = []
costs = []
position_constraints = []

for k, v in _vars.items():
  costs += lpSum([salaries["Pos"][k][i] * _vars[k][i] for i in v])
  rewards += lpSum([pro["Pos"][k][i] * _vars[k][i] for i in v])
  prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available[k]

prob += lpSum(rewards)
prob += lpSum(costs) <= salary_cap

KeyError: ignored

In [31]:
df2[df2.Pos=="QB"].head(10)

,Player,Pos,Salary,Team,Opp,Own,Ceiling,Mean,STD,0,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
24,Sam Ehlinger,QB,4000,IND,TEN,0.0,0.740,0.0,0.565685,0.556176,...,0.333473,-0.745189,2.171958,-0.789111,-0.190787,0.376864,-0.198494,-0.180115,-0.651504,-0.429372
25,Logan Woodside,QB,4000,TEN,IND,0.0,1.496,0.0,1.074802,-0.563261,...,0.012789,-0.904861,-0.471758,-1.213889,0.320731,1.046460,0.209373,-0.122250,0.702770,-0.857113
26,Nathan Peterman,QB,4000,CHI,NYG,0.0,1.560,0.0,1.202082,-1.632738,...,0.937152,-2.541058,-0.384287,-0.105626,-0.230357,-1.527111,1.341125,0.195383,2.360535,0.713435
27,Jarrett Stidham,QB,4500,LV,DEN,0.0,5.008,0.0,2.160583,2.518163,...,2.193348,-1.388737,-1.158322,-0.926588,-3.437119,-2.333345,-4.220229,2.597402,-0.874584,-0.321936
28,Matt Barkley,QB,4000,BUF,BAL,0.0,6.556,0.0,3.736003,4.146803,...,-1.279493,2.603838,-5.497113,-0.690084,-2.662130,1.671351,0.403940,1.258467,-1.944528,2.409907
29,Chris Streveler,QB,4000,NYJ,PIT,0.0,4.464,0.0,3.883242,-9.403437,...,0.671557,-1.826702,-4.064800,5.444836,-3.040461,4.548565,1.728489,6.590017,-4.242903,-7.784164
30,Trace McSorley,QB,4000,ARI,CAR,0.0,4.960,0.0,3.941387,0.555553,...,3.523940,5.603197,1.276421,4.941864,3.574625,-3.657092,0.858135,2.033127,-1.349301,-0.394474
31,Phillip Walker,QB,4800,CAR,ARI,0.0,6.568,0.0,4.154127,1.948281,...,3.031856,2.879704,-2.686504,0.326721,10.145187,-2.742512,0.727316,-1.515196,3.740750,1.751754
32,Josh Rosen,QB,4000,CLE,ATL,0.0,9.660,0.0,4.450180,0.801864,...,-6.418672,2.726916,-5.279455,-3.368933,-0.039841,1.748377,4.351223,3.699592,-9.727313,7.458412
33,Jordan Love,QB,4600,GB,NE,0.0,9.988,0.0,5.154117,-2.551459,...,4.606945,11.584190,2.153136,0.404132,3.449094,0.725011,-9.759037,-2.366258,-5.080883,-0.277560


In [40]:
salaries = {}
pro = {}
for position in df2.Pos.unique():
  available_pos = df2[df2.Pos == "Pos"]
  salary = list(available_pos[["Player", "Salary"]].set_index("Player").to_dict().values())[0]
  proj = list(available_pos[["Player", "Mean"]].set_index("Player").to_dict().values())[0]
  salaries["Pos"] = salary
  pro["Pos"] = proj

pos_num_available = {
    "QB": 1,
    "RB": 3,
    "WR": 4,
    "TE": 2,
    "FLEX": 1,
    "DST": 1
}

pos_flex = {
    "QB": 0,
    "RB": 1,
    "WR": 1,
    "TE": 1,
    "FLEX": 0,
    "DST": 0
}

pos_flex_available = 5



In [41]:
SALARY_CAP = 50000

In [43]:
_vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in pro.items()}

In [46]:
prob = LpProblem("Fantasy", LpMaximize)
rewards = []
costs = []
position_constraints = []

for k, v in _vars.items():
  costs += lpSum([salaries[k][i] * _vars[k][i] for i in v])
  rewards += lpSum([pro[k][i] * _vars[k][i] for i in v])
  prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available[k]
  prob += lpSum([pos_flex[k] * _vars[k][i] for i in v]) <= pos_flex_available

prob += lpSum(rewards)
prob += lpSum(costs) <= SALARY_CAP

KeyError: ignored